# MTA Tracker - Real-Time Transit Data Analysis

This notebook fetches and analyzes data from the MTA (Metropolitan Transportation Authority) API. Follow the steps below to set up your environment and explore the transit data.

## Step 1: Environment Setup

If you're getting "command not found: python", you need to install Python or use the correct command. On macOS:

### Option A: Using Homebrew (Recommended)
```bash
# Install Homebrew if not installed
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

# Install Python 3
brew install python3

# Verify installation
python3 --version

# Create a virtual environment
python3 -m venv mta_env
source mta_env/bin/activate

# Install required packages
pip install requests jupyter pandas matplotlib seaborn protobuf
```

### Option B: Using Python 3 (if already installed)
```bash
# Try using python3 instead of python
python3 MTA_Tracker.ipynb

# Or run Jupyter directly
jupyter notebook
```

### Running This Notebook
After setting up, run:
```bash
jupyter notebook MTA_Tracker.ipynb
```

## Quick Fix: Setup with Homebrew

If you're having issues, run these commands in your terminal to set up properly:

```bash
# 1. Install Python 3 with Homebrew
brew install python3

# 2. Verify Python installation
python3 --version

# 3. Navigate to your project
cd ~/Desktop/Coding\ Projects/MTATracker

# 4. Create a fresh virtual environment
python3 -m venv mta_env

# 5. Activate the environment
source mta_env/bin/activate

# 6. Upgrade pip
pip install --upgrade pip

# 7. Install all required packages at once
pip install requests protobuf pandas matplotlib seaborn

# 8. Start Jupyter from within the environment
jupyter notebook MTA_Tracker.ipynb
```

**Key Points:**
- Use `python3` (not `python`)
- Always activate your virtual environment before running the notebook
- When Jupyter starts, it will use the correct Python from your virtual environment

## Step 2: Import Required Libraries

We'll use these libraries to fetch, process, and analyze MTA data:

In [168]:
import requests
import json
from datetime import datetime
from typing import Optional, Dict, Any
import logging
import pandas as pd

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


In [169]:
class MTATracker:
    """
    MTA Tracker - A utility to fetch and process MTA transit data
    
    This class connects to the Metropolitan Transportation Authority (MTA) real-time
    feed and retrieves GTFS-realtime protobuf data. GTFS stands for General Transit Feed
    Specification - it's the standard format for transit agency data.
    
    The MTA provides three types of real-time data:
    1. VEHICLE positions (current location and status of buses/trains)
    2. TRIP UPDATES (delays and changes to scheduled trips)  
    3. ALERTS (service advisories and disruptions)
    
    All data comes in protobuf (Protocol Buffer) format - a compact binary format.
    """
    
    # API Configuration - URL endpoint for MTA's real-time GTFS feed
    # The URL contains: gtfs = General Transit Feed Specification
    BASE_URL = "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs"
    
    # TIMEOUT = how long to wait for the API response before giving up (in seconds)
    TIMEOUT = 30
    
    def __init__(self):
        """
        Initialize the MTA Tracker.
        
        This sets up:
        - A requests.Session() to reuse the same connection
        - last_update: timestamp of when we last fetched data
        - data: stores the raw binary protobuf data from the API
        """
        self.session = requests.Session()  # HTTP session to reuse connection
        self.last_update = None             # When did we last fetch data?
        self.data = None                    # Raw protobuf bytes from API
        
    def fetch_data(self) -> Optional[bytes]:
        """
        Fetch GTFS data from the MTA API
        
        Returns:
            bytes: Raw response data from the API
        """
        try:
            logger.info(f"Fetching data from MTA API: {self.BASE_URL}")
            
            response = self.session.get(
                self.BASE_URL,
                timeout=self.TIMEOUT
            )
            response.raise_for_status()
            
            self.last_update = datetime.now()
            self.data = response.content
            
            logger.info(f"Successfully fetched {len(self.data)} bytes of data")
            return self.data
            
        except requests.exceptions.Timeout:
            logger.error("Request timed out")
            return None
        except requests.exceptions.ConnectionError:
            logger.error("Failed to connect to MTA API")
            return None
        except requests.exceptions.HTTPError as e:
            logger.error(f"HTTP Error: {e.response.status_code}")
            return None
        except Exception as e:
            logger.error(f"Error fetching data: {str(e)}")
            return None
    
    def parse_data(self) -> Optional[Dict[str, Any]]:
        """
        Parse the fetched data (currently stores raw bytes)
        
        Note: MTA GTFS data is in Protocol Buffer format.
        Future: Implement protobuf parsing
        
        Returns:
            dict: Parsed data structure
        """
        if self.data is None:
            logger.warning("No data available to parse")
            return None
        
        try:
            # Placeholder for data parsing logic
            logger.info("Data ready for processing")
            
            parsed_data = {
                "raw_bytes": len(self.data),
                "timestamp": self.last_update.isoformat() if self.last_update else None,
                "status": "fetched"
            }
            
            return parsed_data
            
        except Exception as e:
            logger.error(f"Error parsing data: {str(e)}")
            return None
    
    def get_status(self) -> Dict[str, Any]:
        """
        Get the current status of the tracker
        
        Returns:
            dict: Status information
        """
        return {
            "last_update": self.last_update.isoformat() if self.last_update else None,
            "data_available": self.data is not None,
            "data_size": len(self.data) if self.data else 0
        }
    
    def close(self):
        """Close the session"""
        self.session.close()
        logger.info("Session closed")

print("✓ MTATracker class defined successfully!")

✓ MTATracker class defined successfully!


In [170]:
# Initialize the tracker
tracker = MTATracker()

# Fetch data from the API
print("🔄 Fetching MTA data...")
data = tracker.fetch_data()

if data:
    print(f"✓ Successfully fetched {len(data)} bytes of data!")
else:
    print("✗ Failed to fetch data from MTA API")

2026-02-02 20:13:23,824 - INFO - Fetching data from MTA API: https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs


🔄 Fetching MTA data...


2026-02-02 20:13:24,013 - INFO - Successfully fetched 176409 bytes of data


✓ Successfully fetched 176409 bytes of data!


In [171]:
# Get tracker status
status = tracker.get_status()
print("📊 Tracker Status:")
print(json.dumps(status, indent=2))

# Get parsed data info
parsed_info = tracker.parse_data()
print("\n📈 Parsed Data Info:")
print(json.dumps(parsed_info, indent=2))

# Display first 100 bytes of raw data (it's binary protobuf format)
print(f"\n🔍 First 100 bytes (raw binary):")
print(tracker.data[:100] if tracker.data else "No data available")

2026-02-02 20:13:24,018 - INFO - Data ready for processing


📊 Tracker Status:
{
  "last_update": "2026-02-02T20:13:24.013110",
  "data_available": true,
  "data_size": 176409
}

📈 Parsed Data Info:
{
  "raw_bytes": 176409,
  "timestamp": "2026-02-02T20:13:24.013110",
  "status": "fetched"
}

🔍 First 100 bytes (raw binary):
b'\n{\n\x031.0\x18\xae\x97\x85\xcc\x06\xca>m\n\x031.0\x12\x0b\n\x011\x12\x06\x10\xae\x97\x85\xcc\x06\x12\x0b\n\x012\x12\x06\x10\xae\x97\x85\xcc\x06\x12\x0b\n\x013\x12\x06\x10\xae\x97\x85\xcc\x06\x12\x0b\n\x014\x12\x06\x10\xae\x97\x85\xcc\x06\x12\x0b\n\x015\x12\x06\x10\xae\x97\x85\xcc\x06\x12\x0b\n\x016\x12\x06\x10\xae\x97\x85\xcc\x06\x12'


## Step 6b: Install GTFS-realtime Packages

Before parsing, let's install the necessary packages (including upgrading pip first):

In [172]:
# Install only necessary packages
import subprocess
import sys

print("Setting up environment...\n")

# Upgrade pip
print("1️⃣  Upgrading pip...")
try:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', '-q'])
    print("   ✓ pip upgraded\n")
except Exception as e:
    print(f"   ⚠ pip upgrade skipped: {e}\n")

# Install protobuf (core requirement)
print("2️⃣  Installing protobuf...")
try:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'protobuf>=3.20', '-q'])
    print("   ✓ protobuf installed\n")
except Exception as e:
    print(f"   ✗ protobuf install failed: {e}\n")
    print("   This is required for parsing!\n")

print("✓ Environment setup complete!")
print("   We'll use pure Python to parse protobuf data.")

Setting up environment...

1️⃣  Upgrading pip...
   ✓ pip upgraded

2️⃣  Installing protobuf...
   ✓ protobuf installed

✓ Environment setup complete!
   We'll use pure Python to parse protobuf data.


In [173]:
import struct

class ProtobufParser:
    """Parse GTFS-realtime protobuf without external dependencies"""
    
    @staticmethod
    def decode_varint(data, pos):
        """Decode protobuf varint"""
        value = 0
        shift = 0
        while pos < len(data):
            byte = data[pos]
            pos += 1
            value |= (byte & 0x7f) << shift
            if (byte & 0x80) == 0:
                break
            shift += 7
        return value, pos
    
    @staticmethod
    def parse_feed(data):
        """Parse FeedMessage"""
        pos = 0
        feed = {"header": {}, "entities": []}
        
        while pos < len(data):
            tag, pos = ProtobufParser.decode_varint(data, pos)
            field_num = tag >> 3
            wire_type = tag & 0x07
            
            if wire_type == 2:  # Length-delimited
                length, pos = ProtobufParser.decode_varint(data, pos)
                field_data = data[pos:pos+length]
                pos += length
                
                if field_num == 1:  # header
                    feed["header"] = ProtobufParser.parse_header(field_data)
                elif field_num == 2:  # entities
                    entity = ProtobufParser.parse_entity(field_data)
                    feed["entities"].append(entity)
        
        return feed
    
    @staticmethod
    def parse_header(data):
        """Parse FeedHeader"""
        header = {}
        pos = 0
        
        while pos < len(data):
            tag, pos = ProtobufParser.decode_varint(data, pos)
            field_num = tag >> 3
            wire_type = tag & 0x07
            
            if wire_type == 2:  # String
                length, pos = ProtobufParser.decode_varint(data, pos)
                value = data[pos:pos+length].decode('utf-8', errors='ignore')
                pos += length
                if field_num == 1:
                    header["version"] = value
            elif wire_type == 0:  # Varint
                value, pos = ProtobufParser.decode_varint(data, pos)
                if field_num == 2:
                    header["timestamp"] = value
                elif field_num == 3:
                    header["incrementality"] = value
        
        return header
    
    @staticmethod
    def parse_entity(data):
        """Parse FeedEntity"""
        entity = {"id": "", "type": "unknown"}
        pos = 0
        
        while pos < len(data):
            tag, pos = ProtobufParser.decode_varint(data, pos)
            field_num = tag >> 3
            wire_type = tag & 0x07
            
            if wire_type == 2:  # Length-delimited
                length, pos = ProtobufParser.decode_varint(data, pos)
                field_data = data[pos:pos+length]
                pos += length
                
                if field_num == 1:  # id
                    entity["id"] = field_data.decode('utf-8', errors='ignore')
                elif field_num == 2:  # trip_update
                    entity["type"] = "trip_update"
                elif field_num == 3:  # vehicle
                    entity["type"] = "vehicle"
                elif field_num == 4:  # alert
                    entity["type"] = "alert"
        
        return entity

# Parse the data
if tracker.data:
    print("🔄 Parsing MTA GTFS-realtime data...\n")
    
    try:
        feed = ProtobufParser.parse_feed(tracker.data)
        
        print("✓ Successfully parsed MTA feed!\n")
        
        # Display header info
        print("📊 Feed Header Information:")
        if feed["header"]:
            if "version" in feed["header"]:
                print(f"   - GTFS Version: {feed['header']['version']}")
            if "timestamp" in feed["header"]:
                from datetime import datetime
                ts = feed["header"]["timestamp"]
                try:
                    readable = datetime.fromtimestamp(ts)
                    print(f"   - Last Update: {readable}")
                except:
                    print(f"   - Timestamp (Unix): {ts}")
            if "incrementality" in feed["header"]:
                inc_type = "FULL_DATASET" if feed["header"]["incrementality"] == 0 else "DIFFERENTIAL"
                print(f"   - Incrementality: {inc_type}")
        
        # Count entity types
        trip_updates = sum(1 for e in feed["entities"] if e["type"] == "trip_update")
        vehicles = sum(1 for e in feed["entities"] if e["type"] == "vehicle")
        alerts = sum(1 for e in feed["entities"] if e["type"] == "alert")
        
        print(f"\n📋 Entities: {len(feed['entities'])} total")
        print(f"   - Trip Updates: {trip_updates}")
        print(f"   - Vehicle Positions: {vehicles}")
        print(f"   - Service Alerts: {alerts}")
        
        # Show samples
        if feed["entities"]:
            print(f"\n📍 Sample Entities (first 5):")
            for i, e in enumerate(feed["entities"][:5]):
                print(f"   {i+1}. ID: {e['id'][:20]}..., Type: {e['type']}")
        
        print(f"\n✓ Data successfully parsed: {len(tracker.data):,} bytes")
        
    except Exception as e:
        print(f"✗ Error parsing: {e}")
        print(f"   Data size: {len(tracker.data):,} bytes")
        import traceback
        traceback.print_exc()
else:
    print("⚠ No data available. Run the 'Fetch Data from MTA API' cell first.")

🔄 Parsing MTA GTFS-realtime data...

✓ Successfully parsed MTA feed!

📊 Feed Header Information:
   - GTFS Version: 1.0
   - Incrementality: DIFFERENTIAL

📋 Entities: 407 total
   - Trip Updates: 0
   - Vehicle Positions: 247
   - Service Alerts: 159

📍 Sample Entities (first 5):
   1. ID: 000001..., Type: vehicle
   2. ID: 000002..., Type: alert
   3. ID: 000003..., Type: vehicle
   4. ID: 000004..., Type: alert
   5. ID: 000005..., Type: vehicle

✓ Data successfully parsed: 176,409 bytes


In [174]:
# Debug: Inspect raw protobuf structure
print("🔍 DEBUGGING: Analyzing protobuf structure\n")

# Let's manually parse a few entities and inspect their binary content
data = tracker.data
pos = 0
entity_count = 0
detailed_entities = []

while pos < len(data) and entity_count < 5:  # Just check first 5 entities
    tag, pos = ProtobufParser.decode_varint(data, pos)
    field_num = tag >> 3
    wire_type = tag & 0x07
    
    if wire_type == 2:  # Length-delimited (our main field)
        length, pos = ProtobufParser.decode_varint(data, pos)
        field_data = data[pos:pos+length]
        pos += length
        
        if field_num == 2:  # entities
            entity_count += 1
            
            print(f"Entity #{entity_count}: ({len(field_data)} bytes)")
            print(f"  Raw hex (first 50 bytes): {field_data[:50].hex()}")
            
            # Parse all fields in this entity
            field_tags = []
            temp_pos = 0
            while temp_pos < len(field_data):
                try:
                    tag2, temp_pos = ProtobufParser.decode_varint(field_data, temp_pos)
                    field_num2 = tag2 >> 3
                    wire_type2 = tag2 & 0x07
                    field_tags.append((field_num2, wire_type2))
                    
                    if wire_type2 == 0:  # varint
                        val, temp_pos = ProtobufParser.decode_varint(field_data, temp_pos)
                    elif wire_type2 == 2:  # length-delimited
                        length2, temp_pos = ProtobufParser.decode_varint(field_data, temp_pos)
                        temp_pos += length2
                    elif wire_type2 == 5:  # 32-bit
                        temp_pos += 4
                except:
                    break
            
            print(f"  Field tags found: {field_tags}")
            print()

print("Field number reference:")
print("  1 = id (string)")
print("  2 = trip_update (nested message)")
print("  3 = vehicle (nested message)")
print("  4 = alert (nested message)")


🔍 DEBUGGING: Analyzing protobuf structure

Entity #1: (89 bytes)
  Raw hex (first 50 bytes): 0a063030303030311a4f0a340a0e3131353535305f312e2e4e3033521a0832303236303230322a0131ca3e140a1030312031
  Field tags found: [(1, 2), (3, 2)]

Entity #2: (78 bytes)
  Raw hex (first 50 bytes): 0a0630303030303222440a340a0e3131353535305f312e2e4e3033521a0832303236303230322a0131ca3e140a1030312031
  Field tags found: [(1, 2), (4, 2)]

Entity #3: (119 bytes)
  Raw hex (first 50 bytes): 0a063030303030331a6d0a340a0e3131353935305f312e2e4e3033521a0832303236303230322a0131ca3e140a1030312031
  Field tags found: [(1, 2), (3, 2)]

Entity #4: (78 bytes)
  Raw hex (first 50 bytes): 0a0630303030303422440a340a0e3131353935305f312e2e4e3033521a0832303236303230322a0131ca3e140a1030312031
  Field tags found: [(1, 2), (4, 2)]

Entity #5: (91 bytes)
  Raw hex (first 50 bytes): 0a063030303030351a510a360a0e3131363030305f312e2e533033521a0832303236303230322a0131ca3e160a1030312031
  Field tags found: [(1, 2), (3, 2)]

Field numbe

In [175]:
# Deep dive into nested message structure
print("🔍 DEEP DIVE: Parsing nested message content\n")

data = tracker.data
pos = 0
entity_num = 0

while pos < len(data) and entity_num < 3:
    tag, pos = ProtobufParser.decode_varint(data, pos)
    field_num = tag >> 3
    wire_type = tag & 0x07
    
    if wire_type == 2:
        length, pos = ProtobufParser.decode_varint(data, pos)
        field_data = data[pos:pos+length]
        pos += length
        
        if field_num == 2:  # entities
            entity_num += 1
            print(f"=== Entity #{entity_num} ===")
            
            # Parse this entity
            ep = 0
            while ep < len(field_data):
                tag2, ep = ProtobufParser.decode_varint(field_data, ep)
                field_num2 = tag2 >> 3
                wire_type2 = tag2 & 0x07
                
                if wire_type2 == 2:  # length-delimited (strings and nested messages)
                    length2, ep = ProtobufParser.decode_varint(field_data, ep)
                    content = field_data[ep:ep+length2]
                    ep += length2
                    
                    if field_num2 == 1:  # id
                        print(f"  ID: {content.decode('utf-8', errors='ignore')}")
                    elif field_num2 in [2, 3, 4]:  # trip_update, vehicle, or alert
                        entity_type = ['trip_update', 'vehicle', 'alert'][field_num2-2]
                        print(f"  Type: {entity_type}")
                        print(f"  Nested message size: {len(content)} bytes")
                        print(f"  Nested hex: {content[:40].hex()}")
                        
                        # Parse nested message fields
                        np = 0
                        nested_fields = []
                        while np < len(content):
                            try:
                                tag3, np = ProtobufParser.decode_varint(content, np)
                                field_num3 = tag3 >> 3
                                wire_type3 = tag3 & 0x07
                                nested_fields.append(f"({field_num3},{wire_type3})")
                                
                                if wire_type3 == 0:
                                    val, np = ProtobufParser.decode_varint(content, np)
                                elif wire_type3 == 2:
                                    len3, np = ProtobufParser.decode_varint(content, np)
                                    np += len3
                                elif wire_type3 == 5:
                                    np += 4
                            except:
                                break
                        
                        print(f"  Nested fields: {nested_fields}")
            print()


🔍 DEEP DIVE: Parsing nested message content

=== Entity #1 ===
  ID: 000001
  Type: vehicle
  Nested message size: 79 bytes
  Nested hex: 0a340a0e3131353535305f312e2e4e3033521a0832303236303230322a0131ca3e140a1030312031
  Nested fields: ['(1,2)', '(2,2)']

=== Entity #2 ===
  ID: 000002
  Type: alert
  Nested message size: 68 bytes
  Nested hex: 0a340a0e3131353535305f312e2e4e3033521a0832303236303230322a0131ca3e140a1030312031
  Nested fields: ['(1,2)', '(3,0)', '(5,0)', '(7,2)']

=== Entity #3 ===
  ID: 000003
  Type: vehicle
  Nested message size: 109 bytes
  Nested hex: 0a340a0e3131353935305f312e2e4e3033521a0832303236303230322a0131ca3e140a1030312031
  Nested fields: ['(1,2)', '(2,2)', '(2,2)']



In [176]:
# IMPROVED PARSER - Focuses on extracting key data (route_id, trip_id)
class BetterProtobufParser:
    """Simplified parser that extracts entity ID, type, and route/trip information"""
    
    @staticmethod
    def decode_varint(data, pos):
        """Decode protobuf varint"""
        value = 0
        shift = 0
        while pos < len(data):
            byte = data[pos]
            pos += 1
            value |= (byte & 0x7f) << shift
            if (byte & 0x80) == 0:
                break
            shift += 7
        return value, pos
    
    @staticmethod
    def parse_feed(data):
        """
        Parse the top-level FeedMessage from raw protobuf bytes.
        
        Returns a dictionary with:
        - header: {"version": "1.0", "timestamp": ...}
        - entities: [list of all vehicles, trip_updates, and alerts]
        
        Algorithm:
        1. Read field tags from the data stream
        2. Extract field_num (what field is this?) and wire_type (what format?)
        3. If it's a length-delimited field (type 2), read its contents
        4. Route to appropriate parser based on field_num
        """
        current_position = 0
        feed_dictionary = {"header": {}, "entities": []}
        
        while current_position < len(data):
            tag, current_position = BetterProtobufParser.decode_varint(data, current_position)
            field_number = tag >> 3       # Get field number (upper bits)
            wire_format_type = tag & 0x07 # Get wire type (lower 3 bits)
            
            if wire_format_type == 2:  # Length-delimited
                length, current_position = BetterProtobufParser.decode_varint(data, current_position)
                field_data = data[current_position:current_position+length]
                current_position += length
                
                if field_number == 1:  # header
                    feed_dictionary["header"] = BetterProtobufParser.parse_header(field_data)
                elif field_number == 2:  # entities
                    entity = BetterProtobufParser.parse_entity(field_data)
                    feed_dictionary["entities"].append(entity)
        
        return feed_dictionary
    
    @staticmethod
    def parse_header(data):
        """Parse FeedHeader"""
        header = {}
        pos = 0
        
        while pos < len(data):
            tag, pos = BetterProtobufParser.decode_varint(data, pos)
            field_num = tag >> 3
            wire_type = tag & 0x07
            
            if wire_type == 2:
                length, pos = BetterProtobufParser.decode_varint(data, pos)
                value = data[pos:pos+length].decode('utf-8', errors='ignore')
                pos += length
                if field_num == 1:
                    header["version"] = value
            elif wire_type == 0:
                value, pos = BetterProtobufParser.decode_varint(data, pos)
                if field_num == 2:
                    header["timestamp"] = value
                elif field_num == 3:
                    header["incrementality"] = value
        
        return header
    
    @staticmethod
    def parse_entity(data):
        """Parse FeedEntity and extract core data"""
        entity = {
            "id": "",
            "type": "unknown",
            "trip_id": "N/A",
            "route_id": "N/A",
            "delay_seconds": "N/A",
            "alert_message": "N/A",
            "affected_routes": "N/A"
        }
        pos = 0
        
        while pos < len(data):
            tag, pos = BetterProtobufParser.decode_varint(data, pos)
            field_num = tag >> 3
            wire_type = tag & 0x07
            
            if wire_type == 2:  # Length-delimited
                length, pos = BetterProtobufParser.decode_varint(data, pos)
                field_data = data[pos:pos+length]
                pos += length
                
                if field_num == 1:  # id
                    entity["id"] = field_data.decode('utf-8', errors='ignore')
                elif field_num == 2:  # trip_update
                    entity["type"] = "trip_update"
                    BetterProtobufParser.extract_trip_update_info(field_data, entity)
                elif field_num == 3:  # vehicle
                    entity["type"] = "vehicle"
                    BetterProtobufParser.extract_vehicle_info(field_data, entity)
                elif field_num == 4:  # alert
                    entity["type"] = "alert"
                    BetterProtobufParser.extract_alert_info(field_data, entity)
        
        return entity
    
    @staticmethod
    def extract_trip_update_info(data, entity):
        """Extract route_id, trip_id, and delay from trip_update"""
        pos = 0
        while pos < len(data):
            try:
                tag, pos = BetterProtobufParser.decode_varint(data, pos)
                field_num = tag >> 3
                wire_type = tag & 0x07
                
                if wire_type == 2:  # Length-delimited
                    length, pos = BetterProtobufParser.decode_varint(data, pos)
                    field_data = data[pos:pos+length]
                    pos += length
                    
                    if field_num == 1:  # trip (nested message)
                        BetterProtobufParser.extract_trip_info(field_data, entity)
                    elif field_num == 2:  # stop_time_updates
                        pass  # Skip for now
                elif wire_type == 0:  # Varint
                    value, pos = BetterProtobufParser.decode_varint(data, pos)
                    if field_num == 3:  # delay
                        entity["delay_seconds"] = str(value)
            except:
                break
    
    @staticmethod
    def extract_vehicle_info(data, entity):
        """Extract route_id and trip_id from vehicle"""
        pos = 0
        while pos < len(data):
            try:
                tag, pos = BetterProtobufParser.decode_varint(data, pos)
                field_num = tag >> 3
                wire_type = tag & 0x07
                
                if wire_type == 2:  # Length-delimited
                    length, pos = BetterProtobufParser.decode_varint(data, pos)
                    field_data = data[pos:pos+length]
                    pos += length
                    
                    if field_num == 1:  # trip (nested message)
                        BetterProtobufParser.extract_trip_info(field_data, entity)
                    elif field_num == 2:  # position
                        pass  # Skip GPS data for now
                elif wire_type == 0:  # Varint
                    value, pos = BetterProtobufParser.decode_varint(data, pos)
            except:
                break
    
    @staticmethod
    def extract_trip_info(data, entity):
        """Extract trip_id and route_id from TripDescriptor"""
        pos = 0
        while pos < len(data):
            try:
                tag, pos = BetterProtobufParser.decode_varint(data, pos)
                field_num = tag >> 3
                wire_type = tag & 0x07
                
                if wire_type == 2:  # String
                    length, pos = BetterProtobufParser.decode_varint(data, pos)
                    value = data[pos:pos+length].decode('utf-8', errors='ignore')
                    pos += length
                    
                    if field_num == 1:  # trip_id
                        entity["trip_id"] = value
                    elif field_num == 3:  # route_id
                        entity["route_id"] = value
                elif wire_type == 0:  # Varint
                    value, pos = BetterProtobufParser.decode_varint(data, pos)
            except:
                break
    
    @staticmethod
    def extract_alert_info(data, entity):
        """Extract alert message and affected routes from Alert"""
        pos = 0
        route_identifier = "N/A"
        
        while pos < len(data):
            try:
                tag, pos = BetterProtobufParser.decode_varint(data, pos)
                field_num = tag >> 3
                wire_type = tag & 0x07
                
                if wire_type == 2:  # Length-delimited
                    length, pos = BetterProtobufParser.decode_varint(data, pos)
                    field_data = data[pos:pos+length]
                    pos += length
                    
                    if field_num == 7:  # description_text - actually contains route identifier
                        route_identifier = field_data.decode('utf-8', errors='ignore').strip()
                elif wire_type == 0:  # Varint
                    value, pos = BetterProtobufParser.decode_varint(data, pos)
            except:
                break
        
        # Set the affected routes to the route identifier found in field 7
        if route_identifier != "N/A":
            entity["affected_routes"] = route_identifier
            entity["alert_message"] = f"Service Alert"  # Generic alert message since MTA doesn't provide text

print("✓ Better parser loaded!")

✓ Better parser loaded!


In [177]:
# Test the new parser
print("🔄 Testing improved parser...\n")

if tracker.data:
    try:
        feed = BetterProtobufParser.parse_feed(tracker.data)
        
        print("✓ Successfully parsed!\n")
        
        # Display header
        print("📊 Feed Header:")
        if feed["header"]:
            print(f"   Version: {feed['header'].get('version', 'N/A')}")
            ts = feed["header"].get('timestamp', 'N/A')
            if ts != 'N/A':
                from datetime import datetime
                readable = datetime.fromtimestamp(ts)
                print(f"   Timestamp: {readable}")
        
        # Count entity types
        trip_updates = sum(1 for e in feed["entities"] if e["type"] == "trip_update")
        vehicles = sum(1 for e in feed["entities"] if e["type"] == "vehicle")
        alerts = sum(1 for e in feed["entities"] if e["type"] == "alert")
        
        print(f"\n📋 Entities: {len(feed['entities'])} total")
        print(f"   - Trip Updates: {trip_updates}")
        print(f"   - Vehicles: {vehicles}")
        print(f"   - Alerts: {alerts}")
        
        # Show first 10 with data
        print(f"\n📌 Sample Entities (first 10):")
        print("-" * 130)
        for i, e in enumerate(feed["entities"][:10]):
            trip = e['trip_id'][:20] if e['trip_id'] != 'N/A' else 'N/A'
            route = e['route_id'][:10] if e['route_id'] != 'N/A' else 'N/A'
            delay = e['delay_seconds'][:10] if e['delay_seconds'] != 'N/A' else 'N/A'
            print(f"{i+1:2}. ID: {e['id'][:15]:15} | Type: {e['type']:12} | Route: {route:10} | Trip: {trip:20} | Delay: {delay:10}")
        
        # Count populated fields
        routes_found = sum(1 for e in feed['entities'] if e['route_id'] != 'N/A')
        trips_found = sum(1 for e in feed['entities'] if e['trip_id'] != 'N/A')
        
        print(f"\n✓ Data extraction complete!")
        print(f"   - Entities with route_id: {routes_found}")
        print(f"   - Entities with trip_id: {trips_found}")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()


🔄 Testing improved parser...

✓ Successfully parsed!

📊 Feed Header:
   Version: 1.0

📋 Entities: 407 total
   - Trip Updates: 0
   - Vehicles: 247
   - Alerts: 159

📌 Sample Entities (first 10):
----------------------------------------------------------------------------------------------------------------------------------
 1. ID: 000001          | Type: vehicle      | Route: 20260202   | Trip: 115550_1..N03R       | Delay: N/A       
 2. ID: 000002          | Type: alert        | Route: N/A        | Trip: N/A                  | Delay: N/A       
 3. ID: 000003          | Type: vehicle      | Route: 20260202   | Trip: 115950_1..N03R       | Delay: N/A       
 4. ID: 000004          | Type: alert        | Route: N/A        | Trip: N/A                  | Delay: N/A       
 5. ID: 000005          | Type: vehicle      | Route: 20260202   | Trip: 116000_1..S03R       | Delay: N/A       
 6. ID: 000006          | Type: alert        | Route: N/A        | Trip: N/A                  | Delay: 

In [178]:
# Re-export data using the new parser with actual extracted data - Save to logs folder
import json
import csv
import os
from datetime import datetime
from pathlib import Path

print("💾 Re-exporting MTA data with extracted fields...\n")

# Create logs directory if it doesn't exist
logs_dir = Path("logs")
logs_dir.mkdir(exist_ok=True)

# Create timestamped subdirectory for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = logs_dir / timestamp
run_dir.mkdir(exist_ok=True)

print(f"📁 Saving to: logs/{timestamp}/\n")

if tracker.data and 'feed' in locals():
    # 1. Save full feed as JSON
    json_file = run_dir / f"mta_feed_{timestamp}_fixed.json"
    with open(json_file, 'w') as f:
        json.dump(feed, f, indent=2, default=str)
    print(f"✓ Saved: {json_file.name} ({len(feed['entities'])} entities)")
    
    # 2. Save entities as CSV with ACTUAL extracted data
    csv_file = run_dir / f"mta_entities_{timestamp}_fixed.csv"
    if feed["entities"]:
        with open(csv_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Entity_ID", "Type", "Route_ID", "Trip_ID", "Delay_Seconds", "Alert_Message", "Affected_Routes"])
            
            for entity in feed["entities"]:
                entity_id = entity.get("id", "N/A")[:50]
                ent_type = entity.get("type", "unknown")
                route = entity.get("route_id", "N/A")
                trip = entity.get("trip_id", "N/A")
                delay = entity.get("delay_seconds", "N/A")
                alert_msg = entity.get("alert_message", "N/A")
                affected_routes = entity.get("affected_routes", "N/A")
                
                writer.writerow([entity_id, ent_type, route, trip, delay, alert_msg, affected_routes])
    
    print(f"✓ Saved: {csv_file.name}")
    
    # 3. Save metadata with updated legend
    meta_file = run_dir / f"mta_metadata_{timestamp}_fixed.txt"
    with open(meta_file, 'w') as f:
        f.write("=" * 70 + "\n")
        f.write("MTA GTFS-REALTIME DATA EXPORT (IMPROVED PARSER)\n")
        f.write("=" * 70 + "\n\n")
        
        f.write("EXPORT TIMESTAMP: " + datetime.now().isoformat() + "\n")
        f.write("DATA SIZE: " + f"{len(tracker.data):,} bytes\n\n")
        
        f.write("FEED HEADER INFORMATION:\n")
        f.write("-" * 70 + "\n")
        if feed["header"]:
            f.write(f"GTFS Version: {feed['header'].get('version', 'N/A')}\n")
            ts = feed['header'].get('timestamp', 'N/A')
            if ts != 'N/A':
                readable = datetime.fromtimestamp(ts)
                f.write(f"Feed Timestamp: {readable}\n")
        
        f.write("\nENTITY BREAKDOWN:\n")
        f.write("-" * 70 + "\n")
        trip_updates = sum(1 for e in feed["entities"] if e["type"] == "trip_update")
        vehicles = sum(1 for e in feed["entities"] if e["type"] == "vehicle")
        alerts = sum(1 for e in feed["entities"] if e["type"] == "alert")
        f.write(f"Total Entities: {len(feed['entities'])}\n")
        f.write(f"  - Trip Updates: {trip_updates}\n")
        f.write(f"  - Vehicles: {vehicles}\n")
        f.write(f"  - Alerts: {alerts}\n\n")
        
        # Data extraction stats
        routes_found = sum(1 for e in feed['entities'] if e['route_id'] != 'N/A')
        trips_found = sum(1 for e in feed['entities'] if e['trip_id'] != 'N/A')
        delays_found = sum(1 for e in feed['entities'] if e['delay_seconds'] != 'N/A')
        
        f.write("DATA EXTRACTION STATISTICS:\n")
        f.write("-" * 70 + "\n")
        f.write(f"Entities with route_id: {routes_found} ({100*routes_found/len(feed['entities']):.1f}%)\n")
        f.write(f"Entities with trip_id: {trips_found} ({100*trips_found/len(feed['entities']):.1f}%)\n")
        f.write(f"Entities with delay: {delays_found} ({100*delays_found/len(feed['entities']):.1f}%)\n\n")
        
        f.write("=" * 70 + "\n")
        f.write("DATA LEGEND / FIELD REFERENCE\n")
        f.write("=" * 70 + "\n\n")
        
        f.write("FILE DESCRIPTIONS:\n")
        f.write("-" * 70 + "\n")
        f.write(f"1. {json_file}\n")
        f.write("   - Full structured JSON export\n")
        f.write("   - Contains all entity data and metadata\n\n")
        
        f.write(f"2. {csv_file}\n")
        f.write("   - Comma-separated values for Excel/Sheets\n")
        f.write("   - Easy to analyze and filter\n\n")
        
        f.write("FIELD DEFINITIONS:\n")
        f.write("-" * 70 + "\n")
        f.write("Entity_ID:     Unique identifier for this transit entity\n")
        f.write("Type:          Entity type: 'vehicle', 'trip_update', or 'alert'\n")
        f.write("Route_ID:      MTA route identifier (e.g., '1', 'A', 'F', etc.)\n")
        f.write("Trip_ID:       Unique identifier for the specific trip\n")
        f.write("Delay_Seconds: Delay in seconds (N/A for vehicles/alerts)\n\n")
        
        f.write("ENTITY TYPES:\n")
        f.write("-" * 70 + "\n")
        f.write("vehicle:\n")
        f.write("  - Real-time vehicle location and status\n")
        f.write("  - Contains: route_id, trip_id, position, bearing\n")
        f.write("  - Count: {}\n\n".format(vehicles))
        
        f.write("trip_update:\n")
        f.write("  - Real-time updates to scheduled trips\n")
        f.write("  - Contains: route_id, trip_id, delay, stop updates\n")
        f.write("  - Count: {}\n\n".format(trip_updates))
        
        f.write("alert:\n")
        f.write("  - Service alerts and announcements\n")
        f.write("  - Contains: alert message, affected routes/agencies\n")
        f.write("  - Count: {}\n\n".format(alerts))
        
        f.write("DATA VALUE LEGEND:\n")
        f.write("-" * 70 + "\n")
        f.write("N/A = Data not available for this entity\n\n")
        
        f.write("CSV FIELD DEFINITIONS:\n")
        f.write("-" * 70 + "\n")
        f.write("Entity_ID:        Unique identifier for this transit entity\n")
        f.write("Type:             Entity type: 'vehicle', 'trip_update', or 'alert'\n")
        f.write("Route_ID:         MTA route identifier (e.g., '1', 'A', 'F')\n")
        f.write("Trip_ID:          Unique identifier for the specific trip\n")
        f.write("Delay_Seconds:    Delay in seconds (for trip_update entities)\n")
        f.write("Alert_Message:    Alert header and description (for alert entities)\n")
        f.write("Affected_Routes:  Routes/agencies affected by alert\n\n")
        
        f.write("HOW TO USE THIS DATA:\n")
        f.write("-" * 70 + "\n")
        f.write("1. Open CSV file in Excel/Google Sheets\n")
        f.write("2. Filter by Type = 'alert' to see all service alerts\n")
        f.write("3. Sort by Route_ID to see specific lines\n")
        f.write("4. Use Alert_Message to understand service issues\n")
        f.write("5. Use Affected_Routes to see which routes are impacted\n")
        
    
    print(f"✓ Saved: {meta_file.name}")
    print(f"\n📁 ALL FILES SAVED TO: logs/{timestamp}/")
    print(f"   1. {json_file.name}")
    print(f"   2. {csv_file.name}")
    print(f"   3. {meta_file.name}")
    
    # Count alerts
    alerts_with_data = sum(1 for e in feed['entities'] if e['type'] == 'alert' and e['alert_message'] != 'N/A')
    print(f"\n✅ Export complete!")
    print(f"   - Alerts with messages: {alerts_with_data}")
    
else:
    print("⚠ No parsed data available")


💾 Re-exporting MTA data with extracted fields...

📁 Saving to: logs/20260202_201324/

✓ Saved: mta_feed_20260202_201324_fixed.json (407 entities)
✓ Saved: mta_entities_20260202_201324_fixed.csv
✓ Saved: mta_metadata_20260202_201324_fixed.txt

📁 ALL FILES SAVED TO: logs/20260202_201324/
   1. mta_feed_20260202_201324_fixed.json
   2. mta_entities_20260202_201324_fixed.csv
   3. mta_metadata_20260202_201324_fixed.txt

✅ Export complete!
   - Alerts with messages: 159


In [179]:

# === REFACTORED: CLEAN PARSING & ALERT MATCHING ===

import csv
import json
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Set, Optional

# === CONSTANTS ===

# Trip ID Format: HHMMSS_LINE..DIRECTIONPATTERN
# Example: 113350_1..S03R → Line 1, Southbound, started at 11:33:50
DIRECTION_MAP = {
    "S": "Southbound",
    "N": "Northbound"
}

# Alert Route Code Format: [LINES][DIRECTION]
# Example: 139S → Lines 1,3,9 going Southbound
ALERT_CODE_DIRECTION_SUFFIX = {"S", "N"}

print("=" * 80)
print("🔧 CLEAN DATA PROCESSING PIPELINE")
print("=" * 80 + "\n")

# ============================================================================
# PART 1: HELPER FUNCTIONS - PARSE TRIP ID
# ============================================================================

def extract_line_from_trip(trip_id: str) -> str:
    """
    Extract subway line from trip ID.
    
    Trip format: HHMMSS_LINE..DIRECTIONPATTERN
    Example: 113350_1..S03R → "1"
    """
    try:
        line_and_direction = trip_id.split("_")[1]
        return line_and_direction[0]  # First character is always the line
    except (IndexError, AttributeError):
        return "N/A"

def extract_direction_from_trip(trip_id: str) -> str:
    """
    Extract direction from trip ID (normalized to full name).
    
    Trip format: HHMMSS_LINE..DIRECTIONPATTERN
    Example: 113350_1..S03R → "Southbound" (S → Southbound)
    """
    try:
        line_and_direction = trip_id.split("_")[1]
        # S or N appears in the string
        direction_char = "S" if "S" in line_and_direction else "N"
        return DIRECTION_MAP.get(direction_char, "Unknown")
    except (IndexError, AttributeError):
        return "Unknown"

def extract_start_time_from_trip(trip_id: str) -> str:
    """
    Extract start time from trip ID.
    
    Trip format: HHMMSS_LINE..DIRECTIONPATTERN
    Example: 113350_1..S03R → "113350" (11:33:50)
    """
    try:
        return trip_id.split("_")[0]
    except (IndexError, AttributeError):
        return "N/A"

def extract_delay_minutes(delay_seconds_str: str) -> Optional[float]:
    """
    Convert delay from seconds string to minutes float.
    Returns None if not available or invalid.
    """
    if delay_seconds_str == "N/A" or not delay_seconds_str:
        return None
    try:
        return int(delay_seconds_str) / 60
    except (ValueError, TypeError):
        return None

# ============================================================================
# PART 2: HELPER FUNCTIONS - PARSE ALERT CODES
# ============================================================================

def extract_lines_from_alert_code(code: str) -> List[str]:
    """
    Extract affected line numbers from alert code.
    
    Alert format: [LINES][DIRECTION]
    Example: 139S → ["1", "3", "9"] (ignoring duplicates/trailing zeros)
    """
    try:
        # Remove direction letter (S or N) from end
        lines_str = code[:-1]
        # Extract individual line numbers, remove duplicates, sort
        lines = sorted(set(lines_str))
        return lines
    except (IndexError, TypeError):
        return []

def extract_direction_from_alert_code(code: str) -> str:
    """
    Extract direction from alert code (normalized).
    
    Alert format: [LINES][DIRECTION]
    Example: 139S → "Southbound"
    """
    try:
        direction_char = code[-1]  # Last character is S or N
        return DIRECTION_MAP.get(direction_char, "Unknown")
    except (IndexError, TypeError):
        return "Unknown"

# ============================================================================
# PART 3: PARSING FUNCTIONS
# ============================================================================

def parse_vehicles_from_csv(csv_file: Path) -> Dict[tuple, Dict]:
    """
    Parse vehicle rows from CSV into structured objects.
    
    Key: (line, direction, start_time) → unique train
    Returns: Dictionary of train objects ready for alert attachment
    """
    trains = {}
    
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['Type'] != 'vehicle':
                continue
            
            trip_id = row.get('Trip_ID', 'N/A')
            if trip_id == 'N/A':
                continue
            
            # Extract trip components
            line = extract_line_from_trip(trip_id)
            direction = extract_direction_from_trip(trip_id)
            start_time = extract_start_time_from_trip(trip_id)
            delay_minutes = extract_delay_minutes(row.get('Delay_Seconds', 'N/A'))
            
            # Use composite key to avoid duplicates
            train_key = (line, direction, start_time)
            
            # Create train object (only if key not already seen)
            if train_key not in trains:
                trains[train_key] = {
                    "line": line,
                    "direction": direction,
                    "start_time": start_time,
                    "trip_id": trip_id,
                    "delay_minutes": delay_minutes,
                    "alerts": set()  # Use set to avoid duplicate alerts
                }
    
    return trains

def parse_alerts_from_csv(csv_file: Path) -> Dict[str, Dict]:
    """
    Parse alert rows from CSV into structured objects.
    
    Key: alert code (e.g., "139S") → alert object
    Returns: Dictionary of alert objects
    """
    alerts = {}
    
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['Type'] != 'alert':
                continue
            
            affected_routes = row.get('Affected_Routes', 'N/A')
            alert_message = row.get('Alert_Message', 'N/A')
            
            # Skip invalid alerts
            if affected_routes == 'N/A' or alert_message == 'N/A':
                continue
            
            # Parse alert structure
            lines = extract_lines_from_alert_code(affected_routes)
            direction = extract_direction_from_alert_code(affected_routes)
            
            # Skip if parsing failed
            if not lines or direction == "Unknown":
                continue
            
            # Store by affected_routes code (unique identifier)
            alerts[affected_routes] = {
                "lines": lines,
                "direction": direction,
                "message": alert_message,
                "affected_routes_code": affected_routes
            }
    
    return alerts

# ============================================================================
# PART 4: MATCHING LOGIC
# ============================================================================

def attach_alerts_to_trains(trains: Dict, alerts: Dict) -> int:
    """
    Attach alerts to trains that match line + direction.
    
    Returns: Total number of alert attachments made
    """
    alert_count = 0
    
    # Index alerts by (line, direction) for faster lookup
    alerts_by_line_direction = defaultdict(list)
    for alert in alerts.values():
        direction = alert["direction"]
        for line in alert["lines"]:
            alerts_by_line_direction[(line, direction)].append(alert)
    
    # Attach alerts to trains
    for train in trains.values():
        line = train["line"]
        direction = train["direction"]
        
        # Lookup alerts for this (line, direction)
        matching_alerts = alerts_by_line_direction.get((line, direction), [])
        
        for alert in matching_alerts:
            train["alerts"].add(alert["message"])
            alert_count += 1
    
    return alert_count

# ============================================================================
# PART 5: OUTPUT FORMATTING
# ============================================================================

def convert_to_json_ready(trains: Dict) -> List[Dict]:
    """
    Convert trains from processing format to JSON-ready format.
    Converts sets to lists, removes internal keys.
    """
    trains_list = []
    
    for train in trains.values():
        trains_list.append({
            "line": train["line"],
            "direction": train["direction"],
            "start_time": train["start_time"],
            "trip_id": train["trip_id"],
            "delay_minutes": train["delay_minutes"],
            "alerts": list(train["alerts"]) if train["alerts"] else []
        })
    
    return trains_list

def build_response_json(trains_list: List[Dict], alerts: Dict) -> Dict:
    """
    Build final JSON response with summary statistics.
    """
    trains_with_alerts = sum(1 for t in trains_list if t["alerts"])
    
    return {
        "last_updated": __import__('datetime').datetime.now().isoformat(),
        "trains": trains_list,
        "alerts": list(alerts.values()),
        "summary": {
            "total_trains": len(trains_list),
            "total_unique_alerts": len(alerts),
            "trains_with_alerts": trains_with_alerts,
            "trains_on_schedule": len(trains_list) - trains_with_alerts
        }
    }

# ============================================================================
# PART 6: MAIN PIPELINE
# ============================================================================

# Find latest log directory
logs_dir = Path("logs")
latest_dir = sorted(logs_dir.glob("*"))[-1]
csv_file = sorted(Path(latest_dir).glob("mta_entities_*_fixed.csv"))[0]

print(f"📂 Reading: {csv_file}\n")

# Step 1: Parse vehicles
print("1️⃣  Parsing vehicles...")
trains = parse_vehicles_from_csv(csv_file)
print(f"   ✓ Extracted {len(trains)} unique trains\n")

# Step 2: Parse alerts
print("2️⃣  Parsing alerts...")
alerts = parse_alerts_from_csv(csv_file)
print(f"   ✓ Extracted {len(alerts)} unique alert codes\n")

# Step 3: Attach alerts to trains
print("3️⃣  Matching alerts to trains...")
total_attachments = attach_alerts_to_trains(trains, alerts)
trains_with_alerts = sum(1 for t in trains.values() if t["alerts"])
print(f"   ✓ Attached {total_attachments} alerts to {trains_with_alerts} trains\n")

# Step 4: Build JSON output
print("4️⃣  Building JSON response...")
trains_list = convert_to_json_ready(trains)
response = build_response_json(trains_list, alerts)
print(f"   ✓ Response built with {len(response['trains'])} trains\n")

# Step 5: Save to file
app_data_file = latest_dir / "app_data.json"
with open(app_data_file, 'w') as f:
    json.dump(response, f, indent=2)
print(f"5️⃣  Saved: {app_data_file}\n")

print("=" * 80)
print("✅ PIPELINE COMPLETE")
print("=" * 80)
print(f"""
📊 Statistics:
   Total trains: {response['summary']['total_trains']}
   Trains on schedule: {response['summary']['trains_on_schedule']}
   Trains affected by alerts: {response['summary']['trains_with_alerts']}
   Total active alerts: {response['summary']['total_unique_alerts']}
""")


🔧 CLEAN DATA PROCESSING PIPELINE

📂 Reading: logs/20260202_201324/mta_entities_20260202_201324_fixed.csv

1️⃣  Parsing vehicles...
   ✓ Extracted 247 unique trains

2️⃣  Parsing alerts...
   ✓ Extracted 139 unique alert codes

3️⃣  Matching alerts to trains...
   ✓ Attached 5387 alerts to 232 trains

4️⃣  Building JSON response...
   ✓ Response built with 247 trains

5️⃣  Saved: logs/20260202_201324/app_data.json

✅ PIPELINE COMPLETE

📊 Statistics:
   Total trains: 247
   Trains on schedule: 15
   Trains affected by alerts: 232
   Total active alerts: 139



In [180]:

# === PART 7: CLEAN API FOR APP QUERIES ===

print("\n" + "=" * 80)
print("🎯 CLEAN QUERY INTERFACE")
print("=" * 80 + "\n")

class MTADataService:
    """
    Clean interface for querying train and alert data.
    Encapsulates all business logic in focused methods.
    """
    
    def __init__(self, response: Dict):
        self.response = response
        self.trains = response["trains"]
        self.alerts = response["alerts"]
        # Build indexes for fast lookups
        self._build_indexes()
    
    def _build_indexes(self):
        """Build lookup indexes for O(1) queries"""
        # Index trains by line
        self.trains_by_line = defaultdict(list)
        for train in self.trains:
            self.trains_by_line[train["line"]].append(train)
        
        # Index trains by (line, direction)
        self.trains_by_line_direction = defaultdict(list)
        for train in self.trains:
            key = (train["line"], train["direction"])
            self.trains_by_line_direction[key].append(train)
        
        # Index alerts by line
        self.alerts_by_line = defaultdict(list)
        for alert in self.alerts:
            for line in alert["lines"]:
                self.alerts_by_line[line].append(alert)
    
    def get_line_status(self, line: str) -> Dict:
        """Get complete status of a subway line"""
        trains = self.trains_by_line.get(line, [])
        alerts = self.alerts_by_line.get(line, [])
        
        return {
            "line": line,
            "total_trains": len(trains),
            "trains_with_alerts": sum(1 for t in trains if t["alerts"]),
            "trains": trains,
            "alerts": alerts
        }
    
    def get_trains_by_direction(self, line: str, direction: str) -> List[Dict]:
        """Get trains on a specific line and direction"""
        key = (line, direction)
        return self.trains_by_line_direction.get(key, [])
    
    def get_delayed_trains(self) -> List[Dict]:
        """Get all trains currently delayed"""
        return [t for t in self.trains if t["delay_minutes"] and t["delay_minutes"] > 0]
    
    def get_trains_with_alerts(self) -> List[Dict]:
        """Get all trains currently affected by alerts"""
        return [t for t in self.trains if t["alerts"]]
    
    def get_train_by_trip_id(self, trip_id: str) -> Optional[Dict]:
        """Lookup single train by trip ID"""
        for train in self.trains:
            if train["trip_id"] == trip_id:
                return train
        return None
    
    def search_alert(self, search_term: str) -> List[Dict]:
        """Search alerts by text"""
        return [
            a for a in self.alerts
            if search_term.lower() in a["message"].lower()
        ]

# Create service instance
service = MTADataService(response)

print("✅ MTADataService ready\n")
print("Available methods:")
print("  • service.get_line_status(line)")
print("  • service.get_trains_by_direction(line, direction)")
print("  • service.get_delayed_trains()")
print("  • service.get_trains_with_alerts()")
print("  • service.get_train_by_trip_id(trip_id)")
print("  • service.search_alert(text)\n")

# ============================================================================
# EXAMPLE QUERIES
# ============================================================================

print("=" * 80)
print("📋 EXAMPLE QUERIES")
print("=" * 80 + "\n")

# Query 1: Line Status
print("1️⃣  Line 1 Status:")
line_1_status = service.get_line_status("1")
print(f"   Total trains: {line_1_status['total_trains']}")
print(f"   With alerts: {line_1_status['trains_with_alerts']}")
print(f"   Active alerts: {len(line_1_status['alerts'])}\n")

# Query 2: Specific direction
print("2️⃣  Line 1 Southbound Trains:")
line_1_south = service.get_trains_by_direction("1", "Southbound")
print(f"   Count: {len(line_1_south)}")
if line_1_south:
    sample = line_1_south[0]
    delay_str = f" (Delayed {sample['delay_minutes']:.0f}min)" if sample['delay_minutes'] else "(On schedule)"
    alert_str = f" - {len(sample['alerts'])} alerts" if sample['alerts'] else ""
    print(f"   Sample: {sample['start_time']} {delay_str}{alert_str}\n")

# Query 3: Delayed trains
print("3️⃣  All Delayed Trains:")
delayed = service.get_delayed_trains()
print(f"   Total: {len(delayed)}")
if delayed:
    for train in delayed[:2]:
        print(f"   • Line {train['line']} {train['direction']}: {train['delay_minutes']:.1f} min late")
print()

# Query 4: Trains with alerts
print("4️⃣  Trains Affected by Alerts:")
affected = service.get_trains_with_alerts()
print(f"   Total: {len(affected)}")
if affected:
    for train in affected[:2]:
        print(f"   • Line {train['line']} {train['direction']} ({train['start_time']})")
        print(f"      {len(train['alerts'])} alert(s)")
print()

# Query 5: Search
print("5️⃣  Search Alerts for 'Service Alert':")
search_results = service.search_alert("Service Alert")
print(f"   Found: {len(search_results)} matches\n")

print("=" * 80)
print("✅ REFACTORED CODE COMPLETE")
print("=" * 80)



🎯 CLEAN QUERY INTERFACE

✅ MTADataService ready

Available methods:
  • service.get_line_status(line)
  • service.get_trains_by_direction(line, direction)
  • service.get_delayed_trains()
  • service.get_trains_with_alerts()
  • service.get_train_by_trip_id(trip_id)
  • service.search_alert(text)

📋 EXAMPLE QUERIES

1️⃣  Line 1 Status:
   Total trains: 40
   With alerts: 40
   Active alerts: 70

2️⃣  Line 1 Southbound Trains:
   Count: 18
   Sample: 116000 (On schedule) - 1 alerts

3️⃣  All Delayed Trains:
   Total: 0

4️⃣  Trains Affected by Alerts:
   Total: 232
   • Line 1 Northbound (115550)
      1 alert(s)
   • Line 1 Northbound (115950)
      1 alert(s)

5️⃣  Search Alerts for 'Service Alert':
   Found: 139 matches

✅ REFACTORED CODE COMPLETE


In [181]:

# === RIDER-FRIENDLY OUTPUT FORMATTING ===

print("\n" + "=" * 80)
print("✨ FORMATTING FOR RIDERS")
print("=" * 80 + "\n")

# ============================================================================
# HELPERS: FORMAT DATA FOR RIDERS
# ============================================================================

def format_24hr_to_12hr(time_24hr: str) -> str:
    """
    Convert GTFS HHMMSS (possibly overflow) into 12-hour clock time.

    Examples:
    "113350" → "11:33 AM"
    "116000" → "12:00 PM"   (minute overflow)
    "246300" → "1:03 AM"    (hour + minute overflow)
    """
    if not time_24hr or not time_24hr.isdigit():
        return time_24hr

    # Pad to 6 digits if needed
    time_24hr = time_24hr.zfill(6)

    try:
        raw_hour = int(time_24hr[:-4])      # everything except MMSS
        raw_minute = int(time_24hr[-4:-2])  # middle two
        raw_second = int(time_24hr[-2:])    # last two

        # Convert entire thing to seconds to normalize overflow
        total_seconds = raw_hour * 3600 + raw_minute * 60 + raw_second

        # Wrap around after 24 hours
        total_seconds %= 24 * 3600

        hour = total_seconds // 3600
        minute = (total_seconds % 3600) // 60

        am_pm = "AM" if hour < 12 else "PM"
        hour_12 = hour % 12 or 12

        return f"{hour_12}:{minute:02d} {am_pm}"

    except ValueError:
        return time_24hr


def enhance_alert_message(alert_message: str, line: str, direction: str) -> str:
    """
    Expand generic alert with line and direction context.
    
    Input: "Service Alert", line="1", direction="Northbound"
    Output: "Service alert affecting Northbound 1 trains"
    """
    if not alert_message or alert_message == "N/A":
        return "Service unavailable"
    
    # If it's generic, enhance with line/direction
    if alert_message.lower() == "service alert":
        return f"Service alert affecting {direction} {line} trains"
    
    # Otherwise return as-is (assume already descriptive)
    return alert_message

def format_train_for_rider(train: Dict) -> Dict:
    """
    Transform backend train object into rider-friendly format.
    
    Removes: trip_id (confusing), keeps: line, direction, readable time
    Enhances: alert messages with context
    """
    formatted_alerts = [
        enhance_alert_message(alert, train["line"], train["direction"])
        for alert in train["alerts"]
    ]
    
    return {
        "line": train["line"],
        "direction": train["direction"],
        "start_time": format_24hr_to_12hr(train["start_time"]),
        "delay_minutes": train["delay_minutes"],
        "alerts": formatted_alerts
    }

# ============================================================================
# BUILD RIDER-FRIENDLY RESPONSE
# ============================================================================

def build_rider_response(response: Dict) -> Dict:
    """
    Transform backend response into rider-friendly format.
    """
    # Format each train for riders
    formatted_trains = [
        format_train_for_rider(train)
        for train in response["trains"]
    ]
    
    return {
        "last_updated": response["last_updated"],
        "trains": formatted_trains,
        "summary": response["summary"]
    }

# Build rider-friendly response
rider_response = build_rider_response(response)

print("📱 Sample Rider-Friendly Trains:\n")

# Show examples
for i, train in enumerate(rider_response["trains"][:5]):
    print(f"{i+1}. Line {train['line']} {train['direction']}")
    print(f"   Departure: {train['start_time']}")
    if train['delay_minutes']:
        print(f"   ⏱️  {train['delay_minutes']:.0f} minutes late")
    else:
        print(f"   ✓ On schedule")
    if train['alerts']:
        for alert in train['alerts']:
            print(f"   ⚠️  {alert}")
    print()

# ============================================================================
# SAVE RIDER-FRIENDLY VERSION
# ============================================================================

rider_json_file = latest_dir / "app_data_rider.json"
with open(rider_json_file, 'w') as f:
    json.dump(rider_response, f, indent=2)

print(f"✅ Saved rider-friendly version: {rider_json_file}\n")

# ============================================================================
# SHOW JSON COMPARISON
# ============================================================================

print("=" * 80)
print("📊 BACKEND vs RIDER-FRIENDLY FORMAT")
print("=" * 80 + "\n")

print("BACKEND FORMAT (raw):")
print(json.dumps(response["trains"][0], indent=2))
print()

print("RIDER-FRIENDLY FORMAT:")
print(json.dumps(rider_response["trains"][0], indent=2))
print()

print("✨ KEY DIFFERENCES:")
print("   ✓ Time: 113350 → 11:33 AM (readable)")
print("   ✓ Hidden: trip_id (backend detail)")
print("   ✓ Alerts: Generic → Specific (with line/direction)")
print("   ✓ No null delays: Either shown or omitted")



✨ FORMATTING FOR RIDERS

📱 Sample Rider-Friendly Trains:

1. Line 1 Northbound
   Departure: 11:55 AM
   ✓ On schedule
   ⚠️  Service alert affecting Northbound 1 trains

2. Line 1 Northbound
   Departure: 11:59 AM
   ✓ On schedule
   ⚠️  Service alert affecting Northbound 1 trains

3. Line 1 Southbound
   Departure: 12:00 PM
   ✓ On schedule
   ⚠️  Service alert affecting Southbound 1 trains

4. Line 1 Northbound
   Departure: 12:03 PM
   ✓ On schedule
   ⚠️  Service alert affecting Northbound 1 trains

5. Line 1 Southbound
   Departure: 12:04 PM
   ✓ On schedule
   ⚠️  Service alert affecting Southbound 1 trains

✅ Saved rider-friendly version: logs/20260202_201324/app_data_rider.json

📊 BACKEND vs RIDER-FRIENDLY FORMAT

BACKEND FORMAT (raw):
{
  "line": "1",
  "direction": "Northbound",
  "start_time": "115550",
  "trip_id": "115550_1..N03R",
  "delay_minutes": null,
  "alerts": [
    "Service Alert"
  ]
}

RIDER-FRIENDLY FORMAT:
{
  "line": "1",
  "direction": "Northbound",
  "sta

In [182]:

# === BUILD TRAIN DATA STRUCTURE FOR YOUR APP ===

print("\n" + "=" * 80)
print("🔧 BUILDING APP DATA STRUCTURES")
print("=" * 80 + "\n")

# Create a structure that's optimized for your app
class MTAData:
    """Stores parsed MTA data in app-friendly format"""
    
    def __init__(self, trains_dict, alerts_list):
        self.trains = trains_dict
        self.alerts = alerts_list
        self.last_updated = datetime.now().isoformat()
    
    def get_line(self, line_num):
        """Get all trains on a specific line"""
        result = []
        for train in self.trains.values():
            if train['line'] == line_num:
                result.append(train)
        return result
    
    def get_line_direction(self, line_num, direction):
        """Get trains on a specific line/direction"""
        result = []
        for train in self.trains.values():
            if train['line'] == line_num and train['direction'] == direction:
                result.append(train)
        return result
    
    def get_line_alerts(self, line_num):
        """Get all active alerts affecting a line"""
        result = []
        for alert in self.alerts:
            if line_num in alert['lines']:
                result.append(alert)
        return result
    
    def to_json(self, file_path=None):
        """Export data as JSON"""
        data = {
            "last_updated": self.last_updated,
            "trains": list(self.trains.values()),
            "alerts": self.alerts,
            "summary": {
                "total_trains": len(self.trains),
                "total_alerts": len(self.alerts),
                "trains_with_delays": sum(1 for t in self.trains.values() if t['delay_minutes']),
                "trains_with_alerts": sum(1 for t in self.trains.values() if t['alerts'])
            }
        }
        
        if file_path:
            with open(file_path, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"✓ Saved to {file_path}")
        
        return data

# Create the data object
mta_data = MTAData(trains, alerts)

print(f"✅ Created MTAData object")
print(f"   - {len(mta_data.trains)} trains loaded")
print(f"   - {len(mta_data.alerts)} active alerts\n")

# === EXAMPLE API CALLS ===

print("📡 EXAMPLE API CALLS FOR YOUR APP\n")

# Example 1: Get status of Line 1
print("1️⃣  Get all trains on Line 1")
line_1_trains = mta_data.get_line('1')
print(f"   Found {len(line_1_trains)} trains on Line 1")
northbound_1 = [t for t in line_1_trains if t['direction'] == 'Northbound']
southbound_1 = [t for t in line_1_trains if t['direction'] == 'Southbound']
print(f"   - Northbound: {len(northbound_1)} trains")
print(f"   - Southbound: {len(southbound_1)} trains\n")

# Example 2: Get alerts for a line
print("2️⃣  Get alerts affecting Line 1")
line_1_alerts = mta_data.get_line_alerts('1')
if line_1_alerts:
    for alert in line_1_alerts[:3]:
        print(f"   ⚠️  {alert['direction']}: {alert['affected_routes_code']} - {alert['message']}")
else:
    print("   ✓ No active alerts on Line 1")
print()

# Example 3: Get specific train
print("3️⃣  Get trains on Line 6 going Northbound")
line_6_north = mta_data.get_line_direction('6', 'Northbound')
if line_6_north:
    for train in line_6_north[:2]:
        status = "🔴 DELAYED" if train['delay_minutes'] else "✓ On schedule"
        print(f"   {train['start_time']} - {status}")
        if train['alerts']:
            for alert in train['alerts']:
                print(f"      ⚠️  {alert}")
else:
    print("   No trains found")
print()

# === EXPORT FOR YOUR APP ===

print("=" * 80)
print("💾 EXPORTING DATA FOR YOUR APPLICATION")
print("=" * 80 + "\n")

# Save as JSON for app consumption
import datetime
app_data_file = latest_dir / "app_data.json"
mta_data.to_json(app_data_file)

print(f"\n✅ Ready for app development!")
print(f"   Use mta_data.get_line() and mta_data.get_line_alerts()")
print(f"   to fetch data in your frontend")



🔧 BUILDING APP DATA STRUCTURES

✅ Created MTAData object
   - 247 trains loaded
   - 139 active alerts

📡 EXAMPLE API CALLS FOR YOUR APP

1️⃣  Get all trains on Line 1
   Found 40 trains on Line 1
   - Northbound: 22 trains
   - Southbound: 18 trains

2️⃣  Get alerts affecting Line 1


TypeError: string indices must be integers, not 'str'

In [ ]:

# === EXAMPLE: Build a simple app interface ===

print("\n" + "=" * 80)
print("📱 EXAMPLE: RIDER APP INTERFACE")
print("=" * 80 + "\n")

def format_train_status(train):
    """Format a single train for display to rider"""
    status = ""
    
    # Line and direction
    status += f"🚆 {train['line']} Train – {train['direction']}\n"
    
    # Delay info
    if train['delay_minutes'] and train['delay_minutes'] > 0:
        status += f"   ⏱️  {train['delay_minutes']:.0f} minutes late\n"
    else:
        status += f"   ✓ On schedule\n"
    
    # Alerts
    if train['alerts']:
        for alert in train['alerts']:
            status += f"   ⚠️  {alert}\n"
    
    status += f"   (Started: {train['start_time']})"
    
    return status

def show_line_status(line_num):
    """Show status of all trains on a line"""
    print(f"\n{'=' * 60}")
    print(f"SUBWAY LINE {line_num}")
    print(f"{'=' * 60}\n")
    
    # Get line alerts
    line_alerts = mta_data.get_line_alerts(line_num)
    if line_alerts:
        print("⚠️  ACTIVE ALERTS ON THIS LINE:")
        for alert in line_alerts:
            print(f"   • {alert['direction']}: {alert['message']}")
        print()
    
    # Show northbound
    northbound = mta_data.get_line_direction(line_num, 'Northbound')
    if northbound:
        print(f"NORTHBOUND: {len(northbound)} trains")
        for train in northbound[:2]:
            print()
            print(format_train_status(train))
        if len(northbound) > 2:
            print(f"\n   ... and {len(northbound) - 2} more northbound trains")
    
    print("\n" + "-" * 60 + "\n")
    
    # Show southbound
    southbound = mta_data.get_line_direction(line_num, 'Southbound')
    if southbound:
        print(f"SOUTHBOUND: {len(southbound)} trains")
        for train in southbound[:2]:
            print()
            print(format_train_status(train))
        if len(southbound) > 2:
            print(f"\n   ... and {len(southbound) - 2} more southbound trains")
    
    print()

# Demo: Show Line 1 and Line 6
show_line_status('1')
show_line_status('6')

print("=" * 80)
print("✅ App data structure ready for development!")
print("=" * 80)



📱 EXAMPLE: RIDER APP INTERFACE


SUBWAY LINE 1

⚠️  ACTIVE ALERTS ON THIS LINE:
   • Southbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Service Alert
   • Northbound: Service Alert
   • Northbound: Service Alert
   • Southbound: Service Alert
   • Southbound: Service Alert
   • Northbound: Ser

In [ ]:

# === INTEGRATION: Ready for Your Frontend ===

print("\n" + "=" * 80)
print("🎯 NEXT STEPS FOR YOUR APPLICATION")
print("=" * 80 + "\n")

print("""
YOUR DATA IS READY! 

The following structures are available:

1️⃣  PYTHON OBJECTS (For Backend Processing)
   • mta_data.trains → Dictionary of all trains
   • mta_data.alerts → List of all alerts
   • mta_data.get_line(line_num) → Get trains on a line
   • mta_data.get_line_direction(line, direction) → Get specific trains
   • mta_data.get_line_alerts(line) → Get alerts on a line

2️⃣  JSON FILES (For Frontend Integration)
   Location: logs/20260202_194910/
   Files:
   • app_data.json → Full train + alert data (ready for API)
   • mta_entities_20260202_194910_fixed.csv → Raw data
   • mta_feed_20260202_194910_fixed.json → Original protobuf parse

3️⃣  EXAMPLE USAGE IN YOUR APP

   # Get Line 1 status
   line_1 = mta_data.get_line('1')
   
   # Filter by direction
   northbound = [t for t in line_1 if t['direction'] == 'Northbound']
   
   # Check for alerts
   alerts = mta_data.get_line_alerts('1')
   
   # Attach alerts to specific train
   for train in line_1:
       if train['alerts']:
           show_warning(f"Train delayed: {train['alerts'][0]}")

4️⃣  FRONTEND INTEGRATION

   a) Load app_data.json in your web/mobile app
   
   b) Display line status:
      ├─ Show number of trains
      ├─ Show alerts (if any)
      └─ Show sample trains + delays
   
   c) Interactive features:
      ├─ Filter by line
      ├─ Filter by direction
      └─ Show active alerts only

5️⃣  API ENDPOINT (If building a backend service)

   GET /api/line/1
   └─ Returns: All trains on Line 1 with alerts
   
   GET /api/line/1/alerts
   └─ Returns: All active alerts on Line 1
   
   GET /api/line/1/Northbound
   └─ Returns: All northbound trains on Line 1

============================================================================

STATISTICS FOR YOUR LATEST RUN
============================================================================
""")

# Calculate statistics
total_trains = len(trains)
total_alerts = len(alerts)
trains_with_alerts = sum(1 for t in trains.values() if t['alerts'])
trains_with_delays = sum(1 for t in trains.values() if t['delay_minutes'])

print(f"""
Total Entities Parsed:           {total_trains + total_alerts}
├─ Trains:                       {total_trains}
└─ Alerts:                       {total_alerts}

Train Status:
├─ On schedule:                  {total_trains - trains_with_delays}
└─ With delays:                  {trains_with_delays}

Alert Impact:
├─ Trains with alerts:           {trains_with_alerts}
└─ Trains unaffected:            {total_trains - trains_with_alerts}

Data Export:
├─ app_data.json:                {app_data_file}
├─ mta_entities_*_fixed.csv:     {latest_dir}/mta_entities_*_fixed.csv
└─ mta_feed_*_fixed.json:        {latest_dir}/mta_feed_*_fixed.json
""")

print("\n" + "=" * 80)
print("✅ PHASE 2 COMPLETE: Data Processing & App Integration Ready")
print("=" * 80)

print("""
YOU'VE SUCCESSFULLY:
  ✓ Fixed the protobuf parser bugs
  ✓ Extracted real vehicle and alert data
  ✓ Built train objects with route/direction info
  ✓ Attached service alerts to affected trains
  ✓ Created app-ready data structures (JSON + Python objects)
  ✓ Demonstrated rider-facing UI patterns

WHAT'S NEXT:
  1. Build your frontend (React, Vue, Svelte, etc.)
  2. Fetch app_data.json from your backend
  3. Render trains by line/direction
  4. Show active alerts with warning icons
  5. Add real-time updates (re-fetch every 30-60 seconds)
""")


## Understanding the Code - Complete Walkthrough

### What This Notebook Does (3 Main Steps)

**1. FETCH** → Get raw binary data from MTA API
```
tracker.fetch_data() → Makes HTTP request → Gets ~200KB protobuf bytes
```

**2. PARSE** → Convert binary protobuf into Python dictionaries  
```
BetterProtobufParser.parse_feed(data) → Decodes binary format → Returns {header, entities}
```

**3. EXPORT** → Save parsed data to readable files (JSON, CSV, TXT)
```
Writes to logs/YYYYMMDD_HHMMSS/ directory with 3 files
```

---

### Variable Names & What They Mean

**Protobuf Parsing Variables:**
- `pos` or `current_position`: Where are we in the binary data? (byte index)
- `tag`: Combined field number + wire type
- `field_num`: Which field is this? (1=header, 2=entities, etc.)
- `wire_type`: How is this field encoded? (0=varint, 2=length-delimited)
- `length`: How many bytes does this field contain?
- `field_data`: The actual bytes of this field

**Entity Variables:**
- `entity`: Python dictionary with parsed data from one entity
- `entity["id"]`: Unique identifier (e.g., "000001")
- `entity["type"]`: What kind? ("vehicle", "trip_update", or "alert")
- `entity["route_id"]`: Which transit line (e.g., "A", "1", "142S")
- `entity["trip_id"]`: Specific trip identifier
- `entity["alert_message"]`: Text of alert (or "Service Alert" for MTA)
- `entity["affected_routes"]`: Which routes are affected

**File Export Variables:**
- `logs_dir`: Path to "logs/" directory
- `timestamp`: Current time as string "20260202_191931"
- `run_dir`: Full path to this run's folder
- `csv_file`, `json_file`, `meta_file`: Paths to output files

---

### Before & After: Alert Data Fix

**THE PROBLEM:**
```python
# Before (broken code was putting route IDs in the wrong column):
000009,alert,N/A,N/A,N/A,137S,N/A
                            ↑
                   Alert_Message column had route data!
```

**THE ROOT CAUSE:**
- MTA protobuf alerts don't have text descriptions
- Field 7 contains the route identifier (142S, 103N, etc.)
- Old code was treating field 7 as an alert message

**THE FIX:**
```python
# Old extract_alert_info():
if field_num == 6:  # header_text
    entity["alert_message"] = extract_text()
elif field_num == 7:  # description_text
    entity["alert_message"] = extract_text()  # ← WRONG! This is route ID

# New extract_alert_info():
if field_num == 7:  # field 7 is actually route identifier in MTA data
    entity["affected_routes"] = extract_text()  # ← CORRECT placement
    entity["alert_message"] = "Service Alert"   # ← Generic message
```

**THE RESULT:**
```python
# After (fixed code puts data in correct columns):
000009,alert,N/A,N/A,N/A,Service Alert,137S
                             ↑           ↑
                    Generic message   Route data (CORRECT!)
```

---

### Key Findings About MTA Data

| Aspect | Finding | Impact |
|--------|---------|--------|
| **Alert Text** | ❌ Not provided by MTA | All alerts show "Service Alert" |
| **Route ID** | ✅ In field 7 | Now correctly extracted |
| **Vehicle Data** | ✅ Complete (route + trip) | 100% extraction rate |
| **Trip Updates** | ❌ None available today | Shows 0 trip_updates |
| **Parsing Method** | Manual protobuf decoder | No external dependencies needed |

---

### How Each Parser Method Works

**`decode_varint(data, pos)` - Read Variable-Length Integers**
```
Protobuf numbers aren't fixed-size. Small numbers use fewer bytes.
Each byte has:
- Bits 0-6: Data bits (7 bits of actual number)
- Bit 7: "More bytes coming" flag

Example:
Byte 1: 0b10000001 = keep going, first 7 bits are "0000001"
Byte 2: 0b00000001 = this is last one, last 7 bits are "0000001"
Result: 0000001 0000001 = 129
```

**`parse_feed(data)` - Top Level**
```
Loop through entire data stream:
1. Read tag (field number + wire type)
2. If field 1 → parse header info
3. If field 2 → parse one entity, add to list
Return {header: {...}, entities: [{...}, {...}, ...]}
```

**`parse_entity(data)` - One Entity**
```
For each entity:
1. Read ID field (field 1)
2. Read type fields (field 2/3/4):
   - If field 2 → it's a trip_update
   - If field 3 → it's a vehicle
   - If field 4 → it's an alert
3. Call appropriate extractor based on type
Return {id, type, trip_id, route_id, delay, alert_message, affected_routes}
```

**`extract_vehicle_info(data, entity)` - Vehicle Data**
```
Vehicles have nested messages:
- Field 1: Trip descriptor (contains route_id, trip_id)
- Field 2: Position (GPS data - we skip)

Call extract_trip_info() to get route + trip
```

**`extract_trip_info(data, entity)` - Trip Details**
```
Trip descriptors have:
- Field 1: trip_id
- Field 3: route_id (note: skips field 2!)

Extract both and add to entity
```

**`extract_alert_info(data, entity)` - Alert Data**
```
Alerts in MTA data have:
- Field 1: Schedule/internal data (we skip)
- Field 3-6: Metadata (we skip)
- Field 7: Route identifier (e.g., "142S", "103N")

Extract field 7 → set affected_routes
Set generic message → "Service Alert"
```

---

### File Structure After Export

```
logs/
├── 20260202_191300/              First run (time-based folder)
│   ├── mta_feed_20260202_191300_fixed.json
│   ├── mta_entities_20260202_191300_fixed.csv
│   └── mta_metadata_20260202_191300_fixed.txt
│
├── 20260202_191931/              Second run (later time)
│   ├── mta_feed_20260202_191931_fixed.json
│   ├── mta_entities_20260202_191931_fixed.csv
│   └── mta_metadata_20260202_191931_fixed.txt
│
└── ... (more timestamped folders)
```

Each run is isolated in its own folder so you can compare data across time.

---

### How to Read the CSV Output

**Open the CSV file in Excel or Google Sheets:**

```
Entity_ID   Type     Route_ID  Trip_ID          Delay  Alert_Message    Affected_Routes
--------    ----     --------  -------          -----  ---------------   ----------------
000001      vehicle  20260202  106550_1..S03R   N/A    N/A               N/A
000002      vehicle  20260202  108950_1..S03R   N/A    N/A               N/A
000003      alert    N/A       N/A              N/A    Service Alert     142S
000004      vehicle  20260202  109150_1..N03R   N/A    N/A               N/A
000005      alert    N/A       N/A              N/A    Service Alert     103N
```

**Rows:**
- Vehicles: Have Route_ID and Trip_ID, rest are N/A
- Alerts: Have only Affected_Routes, rest are N/A
- Trip_Updates: Would have Route_ID, Trip_ID, and Delay_Seconds

**To filter for alerts only in Excel:**
1. Click Data → Filter
2. Click Route_ID dropdown
3. Uncheck to show only when empty
4. Now you see only the 175 alerts



In [ ]:
    @staticmethod
    def parse_vehicle(data, entity):
        """Extract vehicle position and trip info"""
        pos = 0
        while pos < len(data):
            try:
                tag, pos = ProtobufParserFixed.decode_varint(data, pos)
                field_num = tag >> 3
                wire_type = tag & 0x07
                
                if wire_type == 2:  # Length-delimited
                    length, pos = ProtobufParserFixed.decode_varint(data, pos)
                    field_data = data[pos:pos+length]
                    pos += length
                    
                    if field_num == 1:  # trip
                        ProtobufParserFixed.parse_trip_descriptor(field_data, entity)
                    elif field_num == 2:  # position
                        # Skip position parsing for now - complex nested format
                        pass
                elif wire_type == 0:  # Varint
                    value, pos = ProtobufParserFixed.decode_varint(data, pos)
            except:
                break

In [ ]:
# Test with wrapped error handling
print("🔄 Testing improved parser with proper error handling...\n")

if tracker.data:
    try:
        feed_fixed = ProtobufParserFixed.parse_feed(tracker.data)
        
        print("✓ Successfully parsed with enhanced parser!\n")
        
        # Display header
        print("📊 Feed Header:")
        if feed_fixed["header"]:
            print(f"   Version: {feed_fixed['header'].get('version', 'N/A')}")
            print(f"   Timestamp: {feed_fixed['header'].get('timestamp', 'N/A')}")
        
        # Show first 10 entities with details
        print(f"\n📋 First 10 Entities (with extracted data):")
        print("-" * 110)
        for i, e in enumerate(feed_fixed["entities"][:10]):
            print(f"{i+1}. ID: {e['id'][:12]:12} | Type: {e['type']:12} | Route: {e['route_id']:10} | Trip: {e['trip_id'][:12]:12}")
        
        # Count populated fields
        routes_found = sum(1 for e in feed_fixed['entities'] if e['route_id'] != 'N/A')
        trips_found = sum(1 for e in feed_fixed['entities'] if e['trip_id'] != 'N/A')
        
        print(f"\n✓ Successfully extracted {len(feed_fixed['entities'])} entities")
        print(f"  - Entities with route_id: {routes_found}")
        print(f"  - Entities with trip_id: {trips_found}")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()


🔄 Testing improved parser with proper error handling...

✗ Error: unpack requires a buffer of 4 bytes


Traceback (most recent call last):
  File "/var/folders/x6/0kbvm0112w745fhgxxv6wp5c0000gn/T/ipykernel_22940/236673850.py", line 6, in <module>
    feed_fixed = ProtobufParserFixed.parse_feed(tracker.data)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/x6/0kbvm0112w745fhgxxv6wp5c0000gn/T/ipykernel_22940/3356058269.py", line 45, in parse_feed
    entity = ProtobufParserFixed.parse_entity(field_data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/x6/0kbvm0112w745fhgxxv6wp5c0000gn/T/ipykernel_22940/3356058269.py", line 108, in parse_entity
    ProtobufParserFixed.parse_vehicle(field_data, entity)
  File "/var/folders/x6/0kbvm0112w745fhgxxv6wp5c0000gn/T/ipykernel_22940/3356058269.py", line 178, in parse_vehicle
    ProtobufParserFixed.parse_position(field_data, entity)
  File "/var/folders/x6/0kbvm0112w745fhgxxv6wp5c0000gn/T/ipykernel_22940/3356058269.py", line 194, in parse_position
    value, pos = ProtobufParserFixed.dec

In [ ]:
# Debug: Deep dive into alert parsing to understand protobuf structure
print("🔍 DEEP DEBUGGING ALERT PROTOBUF STRUCTURE\n")

# Let's manually parse the first alert to see what's really in the protobuf
if tracker.data:
    # Parse and get first alert entity data
    data = tracker.data
    pos = 0
    first_alert_data = None
    alert_count = 0
    
    # Find the first alert in raw feed
    while pos < len(data) and alert_count == 0:
        tag, pos = BetterProtobufParser.decode_varint(data, pos)
        field_num = tag >> 3
        wire_type = tag & 0x07
        
        if wire_type == 2:
            length, pos = BetterProtobufParser.decode_varint(data, pos)
            field_data = data[pos:pos+length]
            pos += length
            
            if field_num == 2:  # FeedEntity
                # Parse this entity to check if it's an alert
                entity_pos = 0
                is_alert = False
                while entity_pos < len(field_data):
                    etag, entity_pos = BetterProtobufParser.decode_varint(field_data, entity_pos)
                    efield_num = etag >> 3
                    ewire_type = etag & 0x07
                    
                    if ewire_type == 2:
                        elength, entity_pos = BetterProtobufParser.decode_varint(field_data, entity_pos)
                        edata = field_data[entity_pos:entity_pos+elength]
                        entity_pos += elength
                        
                        if efield_num == 4:  # Alert field
                            is_alert = True
                            alert_count += 1
                            print(f"Found Alert #{alert_count}")
                            print(f"Raw alert data length: {len(edata)} bytes")
                            print(f"Raw hex (first 100 bytes): {edata[:100].hex()}")
                            print()
                            
                            # Parse this alert message to see all fields
                            print("Alert Message Fields:")
                            print("-" * 70)
                            alert_pos = 0
                            field_map = {
                                1: "active_period",
                                2: "informed_entity",
                                3: "cause",
                                4: "effect", 
                                5: "url",
                                6: "header_text",
                                7: "description_text"
                            }
                            
                            while alert_pos < len(edata):
                                try:
                                    atag, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                    afield_num = atag >> 3
                                    awire_type = atag & 0x07
                                    
                                    field_name = field_map.get(afield_num, f"field_{afield_num}")
                                    
                                    if awire_type == 2:  # Length-delimited
                                        alength, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                        avalue = edata[alert_pos:alert_pos+alength]
                                        alert_pos += alength
                                        
                                        if afield_num in [1, 3, 4, 5]:  # Complex types, skip
                                            print(f"  {afield_num} ({field_name}): [binary data, {alength} bytes]")
                                        elif afield_num in [6, 7]:  # Text fields
                                            text = avalue.decode('utf-8', errors='ignore')
                                            print(f"  {afield_num} ({field_name}): '{text}'")
                                        elif afield_num == 2:  # informed_entity
                                            print(f"  {afield_num} ({field_name}): [nested message, {alength} bytes]")
                                            # Parse this nested message
                                            inform_pos = 0
                                            while inform_pos < len(avalue):
                                                itag, inform_pos = BetterProtobufParser.decode_varint(avalue, inform_pos)
                                                ifield_num = itag >> 3
                                                iwire_type = itag & 0x07
                                                
                                                if iwire_type == 2:
                                                    ilength, inform_pos = BetterProtobufParser.decode_varint(avalue, inform_pos)
                                                    ivalue = avalue[inform_pos:inform_pos+ilength]
                                                    inform_pos += ilength
                                                    
                                                    ifield_map = {1: "agency_id", 2: "route_id"}
                                                    ifield_name = ifield_map.get(ifield_num, f"field_{ifield_num}")
                                                    itext = ivalue.decode('utf-8', errors='ignore')
                                                    print(f"       └─ {ifield_num} ({ifield_name}): '{itext}'")
                                except:
                                    break
                            break
        elif wire_type == 0:
            value, pos = BetterProtobufParser.decode_varint(data, pos)


🔍 DEEP DEBUGGING ALERT PROTOBUF STRUCTURE

Found Alert #1
Raw alert data length: 70 bytes
Raw hex (first 100 bytes): 0a360a0e3130383935305f312e2e533033521a0832303236303230322a0131ca3e160a10303120313830392b203234322f53465410011803182628aff884cc063a0431343253

Alert Message Fields:
----------------------------------------------------------------------
  1 (active_period): [binary data, 54 bytes]
  7 (description_text): '142S'


In [ ]:
# Debug: Parse field 1 (active_period) to understand the protobuf structure
print("\n\n🔍 PARSING ALERT ACTIVE_PERIOD (Field 1)\n")

# Re-find the first alert
if tracker.data:
    data = tracker.data
    pos = 0
    
    while pos < len(data):
        tag, pos = BetterProtobufParser.decode_varint(data, pos)
        field_num = tag >> 3
        wire_type = tag & 0x07
        
        if wire_type == 2:
            length, pos = BetterProtobufParser.decode_varint(data, pos)
            field_data = data[pos:pos+length]
            pos += length
            
            if field_num == 2:  # FeedEntity
                entity_pos = 0
                found_alert = False
                while entity_pos < len(field_data):
                    etag, entity_pos = BetterProtobufParser.decode_varint(field_data, entity_pos)
                    efield_num = etag >> 3
                    ewire_type = etag & 0x07
                    
                    if ewire_type == 2:
                        elength, entity_pos = BetterProtobufParser.decode_varint(field_data, entity_pos)
                        edata = field_data[entity_pos:entity_pos+elength]
                        entity_pos += elength
                        
                        if efield_num == 4:  # Alert field
                            # Parse ALL fields in alert
                            alert_pos = 0
                            all_fields = []
                            
                            while alert_pos < len(edata):
                                try:
                                    atag, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                    afield_num = atag >> 3
                                    awire_type = atag & 0x07
                                    
                                    if awire_type == 2:  # Length-delimited
                                        alength, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                        avalue = edata[alert_pos:alert_pos+alength]
                                        alert_pos += alength
                                        all_fields.append((afield_num, "LENGTH-DELIMITED", alength, avalue))
                                    elif awire_type == 0:  # Varint
                                        avalue, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                        all_fields.append((afield_num, "VARINT", avalue, None))
                                except:
                                    break
                            
                            print("ALL FIELDS IN FIRST ALERT:")
                            for field_num, wtype, val, data_bytes in all_fields:
                                if wtype == "LENGTH-DELIMITED":
                                    try:
                                        text = data_bytes.decode('utf-8', errors='ignore')
                                        if len(text) < 100:
                                            print(f"  Field {field_num}: {text}")
                                        else:
                                            print(f"  Field {field_num}: {text[:100]}... (truncated)")
                                    except:
                                        print(f"  Field {field_num}: [binary data, {len(data_bytes)} bytes]")
                                else:
                                    print(f"  Field {field_num}: {val}")
                            
                            found_alert = True
                            break
                if found_alert:
                    break




🔍 PARSING ALERT ACTIVE_PERIOD (Field 1)

ALL FIELDS IN FIRST ALERT:
  Field 1: 
108950_1..S03R20260202*1>
01 1809+ 242/SFT
  Field 3: 38
  Field 5: 1770077231
  Field 7: 142S


In [ ]:
# Debug: Check multiple alerts to understand the pattern
print("\n\n🔍 CHECKING MULTIPLE ALERTS TO UNDERSTAND PATTERN\n")

if tracker.data:
    data = tracker.data
    pos = 0
    alert_num = 0
    
    while pos < len(data) and alert_num < 3:
        tag, pos = BetterProtobufParser.decode_varint(data, pos)
        field_num = tag >> 3
        wire_type = tag & 0x07
        
        if wire_type == 2:
            length, pos = BetterProtobufParser.decode_varint(data, pos)
            field_data = data[pos:pos+length]
            pos += length
            
            if field_num == 2:  # FeedEntity
                entity_pos = 0
                while entity_pos < len(field_data):
                    etag, entity_pos = BetterProtobufParser.decode_varint(field_data, entity_pos)
                    efield_num = etag >> 3
                    ewire_type = etag & 0x07
                    
                    if ewire_type == 2:
                        elength, entity_pos = BetterProtobufParser.decode_varint(field_data, entity_pos)
                        edata = field_data[entity_pos:entity_pos+elength]
                        entity_pos += elength
                        
                        if efield_num == 4:  # Alert field
                            alert_num += 1
                            print(f"Alert #{alert_num}:")
                            
                            # Parse this alert
                            alert_pos = 0
                            field_7_value = ""
                            
                            while alert_pos < len(edata):
                                try:
                                    atag, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                    afield_num = atag >> 3
                                    awire_type = atag & 0x07
                                    
                                    if awire_type == 2:
                                        alength, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                        avalue = edata[alert_pos:alert_pos+alength]
                                        alert_pos += alength
                                        try:
                                            text = avalue.decode('utf-8', errors='ignore')
                                            print(f"  Field {afield_num} (text): '{text[:50]}'")
                                            if afield_num == 7:
                                                field_7_value = text
                                        except:
                                            print(f"  Field {afield_num}: [binary, {alength} bytes]")
                                    elif awire_type == 0:
                                        avalue, alert_pos = BetterProtobufParser.decode_varint(edata, alert_pos)
                                        print(f"  Field {afield_num} (int): {avalue}")
                                except:
                                    break
                            print()
        elif wire_type == 0:
            value, pos = BetterProtobufParser.decode_varint(data, pos)

print("➡️  CONCLUSION: Field 7 contains route identifiers, NOT alert descriptions!")
print("    Alerts don't have descriptive text - just route identifiers")




🔍 CHECKING MULTIPLE ALERTS TO UNDERSTAND PATTERN

Alert #1:
  Field 1 (text): '
108950_1..S03R20260202*1>
01 1809+ 242/SFT'
  Field 3 (int): 38
  Field 5 (int): 1770077231
  Field 7 (text): '142S'

Alert #2:
  Field 1 (text): '
109150_1..N03R20260202*1>
01 1811+ SFT/242'
  Field 3 (int): 37
  Field 4 (int): 1
  Field 5 (int): 1770077213
  Field 7 (text): '103N'

Alert #3:
  Field 1 (text): '
109350_1..S03R20260202*1>
01 1813+ 242/SFT'
  Field 3 (int): 37
  Field 4 (int): 1
  Field 5 (int): 1770077244
  Field 7 (text): '139S'

➡️  CONCLUSION: Field 7 contains route identifiers, NOT alert descriptions!
    Alerts don't have descriptive text - just route identifiers


In [ ]:
# Close the tracker session
tracker.close()
print("✓ Session closed successfully!")

2026-02-02 19:01:44,539 - INFO - Session closed


✓ Session closed successfully!


## Step 7: Save and Export Data

Let's save the parsed MTA data to files you can download and analyze later:

In [ ]:
import json
import csv
from datetime import datetime

print("💾 Saving MTA data to files...\n")

if tracker.data and 'feed' in locals():
    # Create timestamp for filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Save full feed as JSON
    json_file = f"mta_feed_{timestamp}.json"
    with open(json_file, 'w') as f:
        json.dump(feed, f, indent=2, default=str)
    print(f"✓ Saved: {json_file}")
    
    # 2. Save entities as CSV
    csv_file = f"mta_entities_{timestamp}.csv"
    if feed["entities"]:
        with open(csv_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Entity_ID", "Type", "Route", "Trip_ID", "Delay_Seconds", "Latitude", "Longitude"])
            
            for entity in feed["entities"]:
                entity_id = entity.get("id", "N/A")[:50]
                ent_type = entity.get("type", "unknown")
                route = entity.get("data", {}).get("route", "N/A")
                trip = entity.get("data", {}).get("trip", "N/A")
                delay = entity.get("data", {}).get("delay", "N/A")
                lat = entity.get("data", {}).get("latitude", "N/A")
                lon = entity.get("data", {}).get("longitude", "N/A")
                
                writer.writerow([entity_id, ent_type, route, trip, delay, lat, lon])
    
    print(f"✓ Saved: {csv_file}")
    
    # 3. Save metadata as text
    meta_file = f"mta_metadata_{timestamp}.txt"
    with open(meta_file, 'w') as f:
        f.write("=" * 70 + "\n")
        f.write("MTA GTFS-REALTIME DATA EXPORT\n")
        f.write("=" * 70 + "\n\n")
        
        f.write("EXPORT TIMESTAMP: " + datetime.now().isoformat() + "\n")
        f.write("DATA SIZE: " + f"{len(tracker.data):,} bytes\n\n")
        
        f.write("FEED HEADER INFORMATION:\n")
        f.write("-" * 70 + "\n")
        if feed["header"]:
            f.write(f"GTFS Version: {feed['header'].get('version', 'N/A')}\n")
            f.write(f"Feed Timestamp: {feed['header'].get('timestamp', 'N/A')}\n")
            f.write(f"Incrementality: {feed['header'].get('incrementality', 'N/A')}\n")
        
        f.write("\nENTITY BREAKDOWN:\n")
        f.write("-" * 70 + "\n")
        trip_updates = sum(1 for e in feed["entities"] if e["type"] == "trip_update")
        vehicles = sum(1 for e in feed["entities"] if e["type"] == "vehicle")
        alerts = sum(1 for e in feed["entities"] if e["type"] == "alert")
        f.write(f"Total Entities: {len(feed['entities'])}\n")
        f.write(f"  - Trip Updates: {trip_updates}\n")
        f.write(f"  - Vehicle Positions: {vehicles}\n")
        f.write(f"  - Service Alerts: {alerts}\n\n")
        
        f.write("=" * 70 + "\n")
        f.write("DATA LEGEND / CYPHER\n")
        f.write("=" * 70 + "\n\n")
        
        f.write("FILE DESCRIPTIONS:\n")
        f.write("-" * 70 + "\n")
        f.write(f"1. {json_file}\n")
        f.write("   Full structured JSON export of all feed data\n")
        f.write("   Format: {header: {...}, entities: [...]}\n\n")
        
        f.write(f"2. {csv_file}\n")
        f.write("   Comma-separated entities with key information\n")
        f.write("   Easy to open in Excel or Google Sheets\n\n")
        
        f.write("FIELD DEFINITIONS:\n")
        f.write("-" * 70 + "\n")
        f.write("Entity_ID: Unique identifier for this transit entity\n")
        f.write("Type: Entity type (trip_update, vehicle, alert)\n")
        f.write("Route: MTA route ID (e.g., '1', 'A', 'F')\n")
        f.write("Trip_ID: Unique trip identifier\n")
        f.write("Delay_Seconds: Delay in seconds (for trip updates)\n")
        f.write("Latitude: Vehicle latitude (for vehicle positions)\n")
        f.write("Longitude: Vehicle longitude (for vehicle positions)\n\n")
        
        f.write("ENTITY TYPES EXPLAINED:\n")
        f.write("-" * 70 + "\n")
        f.write("trip_update:\n")
        f.write("  - Real-time updates about scheduled trips\n")
        f.write("  - Includes: Route ID, delay information, stop updates\n")
        f.write("  - Use: Track schedule changes and delays\n\n")
        
        f.write("vehicle:\n")
        f.write("  - Real-time location of transit vehicles\n")
        f.write("  - Includes: Route, latitude, longitude, bearing\n")
        f.write("  - Use: Track vehicle positions on map\n\n")
        
        f.write("alert:\n")
        f.write("  - Service alerts and notifications\n")
        f.write("  - Includes: Alert messages, affected routes\n")
        f.write("  - Use: Inform users of service changes\n\n")
        
        f.write("HOW TO USE THIS DATA:\n")
        f.write("-" * 70 + "\n")
        f.write(f"1. Open {csv_file} in Excel/Google Sheets for quick overview\n")
        f.write(f"2. Use {json_file} for programmatic access to full data\n")
        f.write("3. Refer to this file for field explanations\n\n")
        
        f.write("EXAMPLE QUERIES:\n")
        f.write("-" * 70 + "\n")
        f.write("Find delayed trips:\n")
        f.write("  - Open CSV, filter Delay_Seconds > 0\n\n")
        
        f.write("Track a specific route (e.g., 'A' line):\n")
        f.write("  - CSV: Filter Route column = 'A'\n")
        f.write("  - JSON: Search for entities with route_id: 'A'\n\n")
        
        f.write("Map vehicle locations:\n")
        f.write("  - Use Latitude + Longitude columns in Google Maps\n")
        f.write("  - Plot as custom locations\n\n")
        
        f.write("NOTES:\n")
        f.write("-" * 70 + "\n")
        f.write("- N/A indicates missing or unavailable data\n")
        f.write("- Timestamps are Unix epoch format (seconds since 1970)\n")
        f.write("- Coordinates use WGS84 (standard GPS)\n")
        f.write("- Data is real-time and changes every 30-60 seconds\n")
        f.write("- Files are timestamped for archival and comparison\n")
    
    print(f"✓ Saved: {meta_file}")
    
    print(f"\n📁 Files saved to current directory:")
    print(f"   1️⃣  {json_file}")
    print(f"       └─ Full data (JSON format)")
    print(f"   2️⃣  {csv_file}")
    print(f"       └─ Entities (CSV - open in Excel)")
    print(f"   3️⃣  {meta_file}")
    print(f"       └─ Legend & data dictionary")
    print(f"\n💡 Start with the metadata file to understand the data!")
    
else:
    print("⚠ No parsed data available to save.")
    print("   Run the parsing cell first to generate feed data.")

💾 Saving MTA data to files...

✓ Saved: mta_feed_20260202_190144.json
✓ Saved: mta_entities_20260202_190144.csv
✓ Saved: mta_metadata_20260202_190144.txt

📁 Files saved to current directory:
   1️⃣  mta_feed_20260202_190144.json
       └─ Full data (JSON format)
   2️⃣  mta_entities_20260202_190144.csv
       └─ Entities (CSV - open in Excel)
   3️⃣  mta_metadata_20260202_190144.txt
       └─ Legend & data dictionary

💡 Start with the metadata file to understand the data!
